In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timezone
import time
import sqlalchemy

# Import Machine Learning Dependencies
#from pathlib import Path
#from collections import Counter
#from sklearn.metrics import balanced_accuracy_score
#from sklearn.metrics import confusion_matrix
#from imblearn.metrics import classification_report_imbalanced

In [4]:
import io
from sqlalchemy import create_engine
import psycopg2 as pg
#load python script that batch loads pandas df to sql
from io import StringIO

In [3]:
pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [5]:
from sodapy import Socrata
from config import MyAppToken
from config import username
from config import password

In [6]:
engine = create_engine("sqlite:///final_project")

In [7]:
#load in the data via API call
client = Socrata('data.austintexas.gov', MyAppToken, username, password)

In [8]:
results = client.get("xwdj-i9he", limit=1160000)

In [9]:
results_df = pd.DataFrame.from_records(results)

In [24]:
results_df.sample(25)

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_zip_code,Created Date,Closed Date,Request Year,Request Month,Open Time,Open Time delta,Total Seconds,Department,Time of Day
401558,"2601 ESPERANZA XING 1211, AUSTIN, TX 78758",7,Phone,2019-07-16T16:22:17.000,3121270.54866258,Closed,Animal - Proper Care,TRAVIS,495D,-97.71726492,...,78758,2019-06-05 13:24:34,2019-07-16 16:22:17,2019,6,41 days 02:57:43,41 days 02:57:43,3553063.0,Animal Services,13
419526,"8402 CROSS PARK DR A, AUSTIN, TX 78754",1,Web,2019-05-07T09:03:36.000,3134442.64706524,Closed,Public Health - Environmental Services - City,TRAVIS,526V,-97.67709164,...,78754,2019-05-04 19:34:51,2019-05-07 09:03:36,2019,5,2 days 13:28:45,2 days 13:28:45,221325.0,Public Health,19
494121,"4511 GREYSTONE DR, AUSTIN, TX 78731",10,Phone,2019-05-10T01:02:00.000,3104675.11016556,Closed,Debris in Street,TRAVIS,494Y,-97.77070287,...,78731,2018-11-10 09:04:22,2019-05-10 01:02:00,2018,11,180 days 15:57:38,180 days 15:57:38,15609458.0,Austin Resource Recovery,9
562596,"1301 QUAIL PARK DR, AUSTIN, TX 78758",4,Phone,2018-12-26T17:28:00.000,3124669.20589467,Closed,Ditch/Driveway Pipe Services,TRAVIS,526A,-97.70733736,...,78758,2018-06-17 07:01:20,2018-12-26 17:28:00,2018,6,192 days 10:26:40,192 days 10:26:40,16626400.0,Watershed Check,7
544487,"DECKER LN & LOYOLA LN, AUSTIN, TX 78724",1,Phone,2018-07-26T09:45:00.000,3150344.79879119,Duplicate (closed),Loose Animal Not Dog,TRAVIS,587D,-97.62798868,...,78724,2018-07-26 05:16:13,2018-07-26 09:45:00,2018,7,0 days 04:28:47,0 days 04:28:47,16127.0,Animal Services,5
614934,"1520 RUTHERFORD LN, AUSTIN, TX 78754",1,Phone,2018-02-12T18:37:10.000,3133220.45999999,Closed,ARR Missed Recycling,TRAVIS,526Y,-97.68097994,...,78754,2018-02-09 12:10:48,2018-02-12 18:37:10,2018,2,3 days 06:26:22,3 days 06:26:22,282382.0,Austin Resource Recovery,12
333876,"500 N IH 35 SVRD SB, AUSTIN, TX 78701",9,Phone,2019-10-09T11:56:20.000,3116638.69999664,Closed,Austin Code - Request Code Officer,TRAVIS,585X,-97.73551976,...,78701,2019-10-09 11:46:45,2019-10-09 11:56:20,2019,10,0 days 00:09:35,0 days 00:09:35,575.0,Austin Code,11
602355,"5809 ARBOR PAVILION LN, AUSTIN, TX 78744",2,Phone,2018-03-19T17:22:21.000,3115963.87006648,Closed,Found Animal - Pick Up,TRAVIS,675B,-97.73965177,...,78744,2018-03-18 18:27:32,2018-03-19 17:22:21,2018,3,0 days 22:54:49,0 days 22:54:49,82489.0,Animal Services,18
365103,"2046 W STASSNEY LN, AUSTIN, TX 78745",5,Phone,2019-08-10T12:52:18.000,3096950.05253372,Closed,Austin Code - Request Code Officer,TRAVIS,643H,-97.79917411,...,78745,2019-08-10 12:42:08,2019-08-10 12:52:18,2019,8,0 days 00:10:10,0 days 00:10:10,610.0,Austin Code,12
584762,"6402 MC NEIL DR, AUSTIN, TX 78729",6,Phone,2018-05-01T17:13:34.000,3109864.49976164,Closed,Austin Code - Request Code Officer,TRAVIS,434Z,-97.7522732,...,78729,2018-05-01 17:05:14,2018-05-01 17:13:34,2018,5,0 days 00:08:20,0 days 00:08:20,500.0,Austin Code,17


In [10]:
len(results_df)

1151553

In [11]:
results_df["Created Date"] = pd.to_datetime(results_df["sr_created_date"])
results_df['Closed Date'] = pd.to_datetime(results_df['sr_closed_date'])

In [12]:
results_df['Request Year'] = pd.DatetimeIndex(results_df['Created Date']).year

In [13]:
results_df['Request Month'] = pd.DatetimeIndex(results_df['Created Date']).month

In [23]:
results_df['Time of Day'] = pd.DatetimeIndex(results_df['Created Date']).hour

In [14]:
#filter the df by Request Year
results_df = results_df[(results_df['Request Year'] == 2018) | (results_df['Request Year'] == 2019)]

In [15]:
results_df['Open Time'] = (results_df['Closed Date'] - results_df['Created Date'])

In [16]:
results_df['Open Time delta'] = pd.to_timedelta(results_df['Open Time'])
results_df['Total Seconds'] = (results_df['Open Time delta']).dt.total_seconds()

In [17]:
results_df = results_df[(results_df['Total Seconds'] < 31540000)]

In [17]:
#median = results_df['Total Seconds'].median(axis=0)

In [18]:
#median

In [18]:
#conditions = [
#    (results_df['Total Seconds'] <= median),
#    (results_df['Total Seconds'] > median)
#    ]
#values = ['Quick Response', 'Slow Response']
#results_df['Response Time'] = np.select(conditions, values)

NameError: name 'median' is not defined

In [20]:
#print(results_df['Total Seconds'].max())

In [21]:
#results_df.loc[results_df['Total Seconds'] == 109367883]

In [22]:
#np.nanpercentile(results_df['Total Seconds'], 99.35) 

In [23]:
#response = results_df['Response Time'].value_counts()
#response

Slow Response     163163
Quick Response    163163
Name: Response Time, dtype: int64

In [19]:
code_types = ['Austin Code - Request Code Officer', 'Code Compliance', 'Code Compliance', 'Construction Ordinance SR', 'Austin Code - Signs/Billboards', 'Austin Code - Short Term Rental Complaint SR', 'Loud Commercial Music', 'Loud Music', 'Austin Code - Coronavirus', 'Austin Code - Short Term Rental (STR) Appointment', 'Austin Code - Construction Rest Break Complaint', 'Austin Code - Construction Ordinance SR', 'Short Term Rental Complaint SR',  'Austin Code - Woodridge Apt']
animal_types = ['Animal - Proper Care', 'Dangerous Animal - Except Dogs', 'Animal Bite',  'Animal Control - Assistance Request', 'Loose Dog', 'Found Animal - Pick Up', 'Loose Animal Not Dog', 'Injured / Sick Animal',  'Found Animal Report - Keep',  'Wildlife Exposure', 'Animal In Vehicle', 'Dangerous/Vicious Dog Investigation', 'Coyote Complaints', 'Animal Trapped in Storm Drain', 'Animal Roadside Sales', 'Loose Animal (not dog)', 'Bat Complaint']
public_types = ['Pothole Repair', 'Obstruction - Urban Forestry', 'Guardrail - New', 'Sidewalk Repair', 'Traffic Signal - Maintenance', 'Traffic Signal - Dig Tess Request',  'Street Light Issue- Address', 'Street Lights New', 'Pavement Failure', 'Sidewalk/Curb Ramp/Route - NEW',  'Alley & Unpaved Street Maintenance',  'Curb/Gutter Repair', 'Bridge Repair', 'Guardrail New/Repair', 'Street Light Issue- Multiple poles/multiple streets', 'Fence/Wall Repair' 'Street Resurfacing', 'Tree Issue ROW/Emergency (PW)', 'Tree Issue ROW/Maintenance (PW)', 'Street Resurfacing Inquiry', 'Guardrail Repair', 'Barricade Request', 'Road Sanding Request']
resource_types = ['ARR Dead Animal Collection', 'ARR Missed Recycling', 'Debris in Street', 'ARR Missed Recycling',  'ARR Missed Yard Trimmings/Compost', 'ARR Brush and Bulk', 'ARR Street Sweeping', 'ARR Missed Yard Trimmings /Organics',  'Ordinance Universal Recycling',  'Ordinance Single-Use Carryout Bags', 'ARR Dumpster', 'Roadway Spillage']
transport_types = ['Lane/Road Closure Notification', 'Traffic Engineering - Jurisdiction Issue', 'Sign - New', 'Bicycle Issues', 'Traffic Calming', 'Traffic Signal - New/Change', 'Shared Micromobility', 'Sign - New',  'Sign - Traffic Sign Emergency', 'Traffic Engineering - General', 'Concerns in the ROW', 'Obstruction in ROW', 'Construction/Permitting- ROW', 'Tree Issue ROW', 'Sign - Traffic Sign Maintenance', 'Road Markings/Striping - Maintenance', 'Parking Violation Enforcement', 'Construction Items in ROW', 'Parking Machine Issue', 'Speed Management', 'Speed Limit - Changes/Signs', 'View Obstruction at Intersection', 'Sign - School Zone Sign Maintenance', 'Sign - Street Name', 'Sign - Parking Sign Maintenance', 'Road Markings/Striping - New', 'School Zone Flasher - Timing/Maintenance', 'ATD - Booting Complaint', 'Filming Permit - ROW', 'Residential Parking Permit Enforcement', 'School Zone - New/Review/Changes', 'EV PIE Station Issues', 'Valet Permit - ROW', 'Vendor Permit - ROW', 'Parking Permit- ROW', 'ATD - Shared Micromobility', 'Newspaper Rack- ROW', 'Construction items in ROW', 'EV PIP Residential Rebate', 'ATD - Pay-by-Phone App', 'Parking Ticket Complaint', 'School Issues - Crossing Guards', 'Dockless Mobility', 'Construction Items- ROW', 'Guardrail - New', 'Emergency Road Closure Report', 'Mowing Medians', 'Emergency Vehicle Preemption Device']
watershed_types = ['Flooding  Current (Non-Emergency)', 'Flood Report', 'Flooding - Storms During Business Hours', 'Flooding - Storms After Business Hours', 'Channels/Creeks/Drainage Easement', 'Flooding - Past', 'Creek & Pond Vegetation Control', 'Storm Drain Pipe Services', 'Standing Water', 'Ditch/Driveway Pipe Services', 'Flooding Current (Non-Emergency)', 'Drainage - Miscellaneous', 'Town Lake Debris Issues', 'Lost Item in Storm Drainage System']
park_types = ['Park Maintenance - Grounds', 'Park Maintenance - Grounds Plumbing Issues', 'Park Maintenance - Pool Issues', 'Park Maintenance - Cemeteries', 'Mowing City Parks', 'Park Maintenance - Building Issues', 'Park Maintenance - Grounds Electrical Issues', 'Park Maintenance - Building Plumbing Issues', 'Park Maintenance - Building Electrical Issues', 'Park Maintenance - Building A/C & Heating Issues', 'Aquatics Hotline Inquiry']
water_types = ['AW - Water Theft Report', 'AW - Water Waste Report', 'Water Waste Report', 'Water Theft Report']
health_types = ['Public Health - Graffiti Abatement', 'Basic Needs - Appointment', 'Basic Needs - Information Referral',  'Public Health - Environmental Services - City', 'Public Health - Environmental Services - County', 'Graffiti Abatement', 'Public Health - Environmental Services - City', 'Heat Advisory', 'HSEM Heat Advisory']
housing_types = ['LeadSmart Program', 'Neighborhood Home Programs']
fire_types = ['AFD - Wildfire Concern / Presentation']

In [20]:
results_df['Code Check'] = results_df['sr_type_desc'].isin(code_types)
results_df['Animal Check'] = results_df['sr_type_desc'].isin(animal_types)
results_df['Public Types'] = results_df['sr_type_desc'].isin(public_types)
results_df['Resource Check'] = results_df['sr_type_desc'].isin(resource_types)
results_df['Transport Check'] = results_df['sr_type_desc'].isin(transport_types)                                                                
results_df['Watershed Check'] = results_df['sr_type_desc'].isin(watershed_types)
results_df['Park Check'] = results_df['sr_type_desc'].isin(park_types)
results_df['Water Check'] = results_df['sr_type_desc'].isin(water_types)
results_df['Health Check'] = results_df['sr_type_desc'].isin(health_types)
results_df['Housing Check'] = results_df['sr_type_desc'].isin(housing_types)
results_df['Fire Check'] = results_df['sr_type_desc'].isin(fire_types)

In [21]:
results_df.loc[results_df["Code Check"] == True, "Department"] = 'Austin Code'
results_df.loc[results_df["Animal Check"] == True, "Department"] = 'Animal Services'
results_df.loc[results_df["Public Types"] == True, "Department"] = 'Public Works'
results_df.loc[results_df["Resource Check"] == True, "Department"] = 'Austin Resource Recovery'
results_df.loc[results_df["Transport Check"] == True, "Department"] = 'Austin Transportation'
results_df.loc[results_df["Watershed Check"] == True, "Department"] = 'Watershed Check'
results_df.loc[results_df["Park Check"] == True, "Department"] = 'Parks and Recreation'
results_df.loc[results_df["Water Check"] == True, "Department"] = 'Austin Water'
results_df.loc[results_df["Health Check"] == True, "Department"] = 'Public Health'
results_df.loc[results_df["Housing Check"] == True, "Department"] = 'Housing and Planning'
results_df.loc[results_df["Fire Check"] == True, "Department"] = 'Austin Fire'

In [27]:
#top_dept = results_df['Department'].value_counts()
#top_dept

In [28]:
#print(results_df.columns)

In [22]:
results_df = results_df.drop(columns=['Code Check', 'Animal Check', 'Public Types','Resource Check','Transport Check','Watershed Check','Park Check','Water Check','Health Check','Housing Check','Fire Check'])

In [25]:
training_df = results_df.drop(columns=['sr_location', 'Open Time', 'sr_location_lat_long', 'sr_location_council_district', 'sr_closed_date', 'sr_location_x', 'sr_status_desc', 'sr_location_county', 'sr_location_map_page', 'sr_location_long', 'sr_updated_date', 'sr_location_lat', 'sr_status_date', 'sr_location_street_name', 'sr_created_date', 'sr_location_street_number', 'sr_location_city', 'sr_location_y', 'sr_location_map_tile', 'Created Date', 'Closed Date', 'Open Time delta'])

In [26]:
training_df = training_df.drop(columns=['sr_type_desc','sr_number'])

In [32]:
#training_df = training_df.drop(columns=[])

In [27]:
training_df = training_df.rename(columns={'sr_method_received_desc':'Report Method', "sr_location_zip_code":"Zip Code"})

In [42]:
training_df = training_df.rename(columns={'Zip Code':'ZipCode'})

In [43]:
training_df.head(10)

,Report Method,ZipCode,Request Year,Request Month,Total Seconds,Department,Time of Day
297854,Phone,78723,2019,12,152190.0,Austin Code,23
297855,Spot311 Interface,78744,2019,12,152934.0,Austin Code,23
297856,Web,78701,2019,12,113327.0,Public Works,23
297857,Spot311 Interface,78701,2019,12,121696.0,Austin Resource Recovery,23
297858,Phone,78701,2019,12,155392.0,Austin Code,23
297859,Phone,78745,2019,12,18479378.0,Animal Services,22
297860,Spot311 Interface,78745,2019,12,155819.0,Austin Code,22
297861,Phone,78746,2019,12,1097446.0,Austin Resource Recovery,22
297862,Spot311 Interface,78745,2019,12,117788.0,Animal Services,22
297863,Spot311 Interface,78660,2019,12,162582.0,Animal Services,22


In [29]:
#read in census data from csv exports from data.census.gov 
mfi_data = "circle/MFI by Zipcode.csv"
pop_data = "circle/Population by Zipcode.csv"

mfi_df = pd.read_csv(mfi_data)
pop_df = pd.read_csv(pop_data)

In [30]:
#drop 0 index row
pop_df = pop_df.drop(0)

In [31]:
mfi_df = mfi_df.drop(0)

In [32]:
#regex to match just zip code 
pop_df['NAME'] = pop_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [33]:
mfi_df['NAME'] = mfi_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [34]:
pop_df = pop_df.rename(columns={'NAME':'ZipCode','B01003_001E':'TotalPopulation'})

In [35]:
pop_df = pop_df.drop(columns=['GEO_ID','B01003_001M'])

In [36]:
mfi_df = mfi_df.rename(columns={'NAME':'ZipCode','S1903_C03_015E':'MedianFamilyIncome'})

In [37]:
mfi_df = mfi_df.drop(columns=['GEO_ID'])

In [40]:
joined_df = mfi_df.join(pop_df.set_index('ZipCode'), on='ZipCode')

In [41]:
joined_df.head()

,ZipCode,MedianFamilyIncome,TotalPopulation
1,78610,88053,38525
2,78613,115556,81943
3,78617,63405,28930
4,78641,108589,72078
5,78652,103846,5374


In [45]:
training_df = training_df.join(joined_df.set_index('ZipCode'), on='ZipCode')

In [46]:
training_df.head()

,Report Method,ZipCode,Request Year,Request Month,Total Seconds,Department,Time of Day,MedianFamilyIncome,TotalPopulation
297854,Phone,78723,2019,12,152190.0,Austin Code,23,73162,35725
297855,Spot311 Interface,78744,2019,12,152934.0,Austin Code,23,52730,50123
297856,Web,78701,2019,12,113327.0,Public Works,23,175993,10848
297857,Spot311 Interface,78701,2019,12,121696.0,Austin Resource Recovery,23,175993,10848
297858,Phone,78701,2019,12,155392.0,Austin Code,23,175993,10848


In [35]:
#training_df.rename(columns={'index':'Index'}, inplace =True)
#command = 'Index serial primary key, Report Method text, Zip Code varchar(5), Request Year int, Total Seconds interval, Response Time text, Department text'


training_df.to_sql('database', con=engine, if_exists='replace')

engine.execute("SELECT * FROM database").fetchall()

[(295408, 'Phone', '78723', 2019, 152190.0, 'Slow Response', 'Austin Code'),
 (295409, 'Spot311 Interface', '78744', 2019, 152934.0, 'Slow Response', 'Austin Code'),
 (295410, 'Web', '78701', 2019, 113327.0, 'Slow Response', 'Public Works'),
 (295411, 'Spot311 Interface', '78701', 2019, 121696.00000000001, 'Slow Response', 'Austin Resource Recovery'),
 (295412, 'Phone', '78701', 2019, 155392.0, 'Slow Response', 'Austin Code'),
 (295413, 'Phone', '78745', 2019, 18479378.0, 'Slow Response', 'Animal Services'),
 (295414, 'Spot311 Interface', '78745', 2019, 155819.0, 'Slow Response', 'Austin Code'),
 (295415, 'Phone', '78746', 2019, 1097446.0, 'Slow Response', 'Austin Resource Recovery'),
 (295416, 'Spot311 Interface', '78745', 2019, 117788.00000000001, 'Slow Response', 'Animal Services'),
 (295417, 'Spot311 Interface', '78660', 2019, 162582.0, 'Slow Response', 'Animal Services'),
 (295418, 'Spot311 Interface', '78757', 2019, 162712.0, 'Slow Response', 'Animal Services'),
 (295419, 'Phone'

Machine Learning Model

In [36]:
# Create our features
X = training_df.drop(columns = ["Response Time"])

# Create dummy variables
X = pd.get_dummies(X)

# Create our target
y = training_df[["Response Time"]]

In [37]:
# Check the balance of our target values
y["Response Time"].value_counts()

Slow Response     163163
Quick Response    163163
Name: Response Time, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [39]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rand_for_class = BalancedRandomForestClassifier(n_estimators = 100, random_state = 1)
rand_for_class.fit(X_train, y_train.values.ravel())

BalancedRandomForestClassifier(random_state=1)

In [40]:
# Calculated the balanced accuracy score
y_prediction = rand_for_class.predict(X_test)
balanced_accuracy_score(y_test, y_prediction)

0.9999877621949727

In [41]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction)

array([[40725,     0],
       [    1, 40856]], dtype=int64)

In [42]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction))

                      pre       rec       spe        f1       geo       iba       sup

Quick Response       1.00      1.00      1.00      1.00      1.00      1.00     40725
 Slow Response       1.00      1.00      1.00      1.00      1.00      1.00     40857

   avg / total       1.00      1.00      1.00      1.00      1.00      1.00     81582



In [43]:
# List the features sorted in descending order by feature importance
features = rand_for_class.feature_importances_
sort_features = np.argsort(features)[::-1]

for i in sort_features:
    print("{}: ({})".format(X.columns.values[i], features[i]))

Total Seconds: (0.8199200955415215)
Department_Austin Code: (0.10911936345980736)
Department_Austin Resource Recovery: (0.022420757230637797)
Department_Austin Transportation: (0.009375291534289414)
Department_Watershed Check: (0.004970995026808016)
Department_Animal Services: (0.004845010574832926)
Report Method_Phone: (0.0042809026538513295)
Department_Public Works: (0.004101226195109957)
Report Method_Spot311 Interface: (0.00363657726094304)
Department_Parks and Recreation: (0.0022850302780916634)
Report Method_Web: (0.00219988062050061)
Department_Public Health: (0.0020719595887028898)
Zip Code_78701: (0.0020654004595478306)
Request Year: (0.0020103580644305016)
Department_Austin Water: (0.0013375763456474098)
Zip Code_78702: (0.000375256733068445)
Zip Code_78741: (0.0003125961411242852)
Zip Code_78758: (0.0002970271629739315)
Zip Code_78703: (0.0002559678811358992)
Zip Code_78753: (0.00024440964171287675)
Zip Code_78705: (0.0002183487559804042)
Zip Code_78749: (0.00019482219832465